In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import numpy as np
import cv2
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

from ssd import build_ssd
%matplotlib inline
from matplotlib import pyplot as plt
import random
import os.path as osp

## Define class and matching index

In [2]:
label_list = ['3m', 'andes', 'cocacola', 'crayola', 'folgers','heineken','hunts','kellogg','kleenex',\
               'kotex','libava','macadamia','milo','mm','pocky','raisins','stax','swissmiss','vanish','viva']
LABEL = dict(zip(label_list, range(len(label_list))))

In [3]:
DATA_ROOT = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/real/"
IMG_ROOT = osp.join(DATA_ROOT, "image")
MASK_ROOT = osp.join(DATA_ROOT, "mask")
img_list = os.listdir(IMG_ROOT)
mask_list = os.listdir(MASK_ROOT)
GT_ROOT = osp.join(DATA_ROOT, "groundtruths_mmfcn")
DT_ROOT = osp.join(DATA_ROOT, "detections_real")
if not osp.exists(GT_ROOT):
    os.makedirs(GT_ROOT)
if not osp.exists(DT_ROOT):
    os.makedirs(DT_ROOT)

## Create ground truth file for mmfcn real data

In [4]:
for data in mask_list:
    mask = cv2.imread(osp.join(MASK_ROOT, data), cv2.IMREAD_GRAYSCALE)
    name = data.split('.')[0]
    x, y, w, h = cv2.boundingRect(mask)
    if (x, y, w, h) != (0, 0, 0, 0):
        label = int(mask[mask!=0][0])-1
        cls = list(LABEL.keys())[list(LABEL.values()).index(label)]
        # <x> <y> <width> <height>
        f = open(osp.join(GT_ROOT, name + ".txt"), "w+")
        ss = cls + " " + str(x) + " " + str(y) + " " + str(w) + " " + str(h)
        f.write(ss)
        f.close() 
    else: # if there is no any object inside
        f = open(osp.join(GT_ROOT, name + ".txt"), "w+")
        f.close()

## SSD detection

In [4]:
net = build_ssd('test', 300, 21)    # initialize SSD, +1 for background
net.load_weights('/media/arg_ws3/5E703E3A703E18EB/research/mm_fcn/ssd/real/real_21.pth')

Loading weights into state dict...
Finished!


/home/arg_ws3/ssd.pytorch/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/arg_ws3/ssd.pytorch/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


## Build SSD300 Network

In [5]:
for img in img_list:
    image = cv2.imread(osp.join(IMG_ROOT, img))

    x = cv2.resize(image, (300, 300)).astype(np.float32)
    x -= (104.0, 117.0, 123.0)
    x = x.astype(np.float32)
    x = x[:, :, ::-1].copy()
    x = torch.from_numpy(x).permute(2, 0, 1)

    xx = Variable(x.unsqueeze(0))     # wrap tensor in Variable
    if torch.cuda.is_available():
        xx = xx.cuda()
    y = net(xx)

    scale = torch.Tensor(image.shape[1::-1]).repeat(2)
    detections = y.data # torch.Size([1, 4, 200, 5]) --> [batch?, class, object, coordinates]
    objs = []
    for i in range(detections.size(1)): # detections.size(1) --> class size
        for j in range(5): # each class choose top 5 predictions
            if detections[0, i, j, 0].cpu().numpy() > 0.1:
                score = detections[0, i, j, 0]
                pt = (detections[0, i, j,1:]*scale).cpu().numpy()
                objs.append([int(pt[0]), int(pt[1]), int(pt[2]-pt[0]+1), int(pt[3]-pt[1]+1), \
                             list(LABEL.keys())[list(LABEL.values()).index(i-1)], score.item()])
    #print(objs)
    f = open(osp.join(DT_ROOT,  img.split('.')[0] + ".txt"), "w+")
    for obj in objs:
        ss = obj[4] + " " + str(obj[5])[1:] + " " + str(obj[0]) + " " + str(obj[1]) +\
        " " + str(obj[2]) + " " + str(obj[3]) + "\n"
        f.write(ss)
    f.close()